In [2]:
!pip install sqlalchemy beautifulsoup4  selenium requests

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from sqlalchemy import create_engine

### Testing for response 200

In [4]:
url = "https://www.aliexpress.com/w/wholesale-laptop.html?spm=a2g0o.home.search.0"
r = requests.get(url)
print(r)

ConnectionError: HTTPSConnectionPool(host='www.aliexpress.com', port=443): Max retries exceeded with url: /w/wholesale-laptop.html?spm=a2g0o.home.search.0 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000022BA9674E50>: Failed to resolve 'www.aliexpress.com' ([Errno 11001] getaddrinfo failed)"))

In [4]:
# Configure Selenium Chromedriver options
options = Options()
options.headless = True
service = Service(executable_path=r'C:\Users\Aise\Kune\chromedriver-win64\chromedriver.exe')

#Initialize the webdriver
driver = webdriver.Chrome(service=service, options=options)

# Define the URL
url = "https://www.aliexpress.com/w/wholesale-laptop.html?spm=a2g0o.home.search.0"

# Use Selenium to open the page

time.sleep(10)

page_source = driver.page_source
driver.quit()

### Extraction Layer

In [5]:
options = Options()
options.headless = True
service = Service(executable_path=r'C:\Users\Aise\Kune\chromedriver-win64\chromedriver.exe')

driver = webdriver.Chrome(service=service, options=options)

laptop_names = []
prices = []
store_names = []
store_link = []
shipping_price = []
extra_discounts = []
item_sold = []
old_price = []
shipping_free_status = []
sales_prices = []

# assuming 
total_page_num = 60

for i in range(1, total_page_num + 1):
    url = f"https://www.aliexpress.com/w/wholesale-laptop.html?page={i}&g=y&SearchText=laptop"
    driver.get(url)
    time.sleep(10)

    # Use BeautifulSoup 
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Adjust the class selectors based on the current web site structure
    laptops = soup.find_all('div', class_ = "list--gallery--C2f2tvm search-item-card-wrapper-gallery")

    for products in laptops:
        # product_name
        product_name = products.find('h3', class_ = "multi--titleText--nXeOvyr").text
        # prices
        price = products.find('div', class_ = "multi--price--1okBCly").text.replace('NGN','').replace(',','')
        # store_name
        store_name = products.find('a', class_ = "cards--storeLink--XkKUQFS").text
        # store_link
        try:
            store_links = products.find('a',"cards--storeLink--XkKUQFS")['href']
        except(TypeError, KeyError):
            store_links = ''
        
        #shipping_price
        try:
            shipping_prices = products.find('span', class_ = "tag--text--1BSEXVh tag--textStyle--3dc7wLU multi--serviceStyle--1Z6RxQ4").text
        except AttributeError:
            shipping_prices = ''
        
        #discount
        try:
            extra_discount = products.find('span', class_ = "tag--text--1BSEXVh tag--textStyle--3dc7wLU multi--superStyle--1jUmObG").text
        except AttributeError:
            extra_discount = ''

        # item sold
        try:
            item_solds = products.find('span', class_ = "multi--trade--Ktbl2jB").text
            item_solds = item_solds.split()[0] if item_solds else ''
        except AttributeError:
            item_solds = ''
        
        # Append
        laptop_names.append(product_name)
        prices.append(price)
        store_names.append(store_name)
        store_link.append(store_links)
        shipping_price.append(shipping_prices)
        extra_discounts.append(extra_discount)
        item_sold.append(item_solds)

        #Extracting additional information
        old_price = []
        former_prices = products.find_all('div', class_= "multi--price-original--1zEQqOK")
        old_price.append(former_prices[0].text if former_prices else '')
        
       
        shipping_free_status = []
        sales_prices = []
driver.quit()

# Creating Dataframes
data = {'name':laptop_names,
        'prices': prices, 
        'store_names': store_names, 
        'store_link': store_link, 
        'shipping_price': shipping_price,
        'extra_discounts': extra_discounts, 
        'item_sold':item_sold, 
        'old_price':old_price
        }

df = pd.DataFrame.from_dict(data,orient = 'index').transpose()

df


,name,prices,store_names,store_link,shipping_price,extra_discounts,item_sold,old_price
0,High-end Lenovo Laptop ThinkPad X1 Carbon 2022...,8195500,HUWEI Factory Store,//www.aliexpress.com/store/836146,Free shipping,,6,
1,Gaming Laptop 15.6 Inch AMD R3 2200U 32GB RAM ...,639134.271304346.61,BSLAY Official Store,//www.aliexpress.com/store/5839012,Free shipping,,3,None
2,2022 Professional Lenovo Gaming Laptop Legion ...,6228580,HUWEI Factory Store,//www.aliexpress.com/store/836146,Free shipping,,4,None
3,VCHANCE 17.3 Inch 1080p 144Hz Gaming Portable ...,324246.72,VCHANCE Official Store,//www.aliexpress.com/store/1102820440,,Extra 5% off with coins,4,None
4,Gaming Laptop HP OMEN 9 13th Gen Intel Core i5...,9263728.79,Blue Ocean Laptop Store,//www.aliexpress.com/store/1102248573,,,2,None
...,...,...,...,...,...,...,...,...
613,"B125XTN01.0 fit 12.5""1366X768 HB125WX1-200 LP1...",65347.3,LCD Source Store,//www.aliexpress.com/store/1102460600,Free shipping,Extra 2% off with coins,,None
614,LP156WFC(SP)(H1) 30PINS 60HZ 1920*1080 Lap...,76429.58,Stars LCD Screen Store,//www.aliexpress.com/store/1102685113,,,1,None
615,LP133X7 N2C2 fit TM133XG-07 Laptop LCD LED Dis...,64664.06,LCD Source Store,//www.aliexpress.com/store/1102460600,Free shipping,Extra 2% off with coins,,None
616,17.3'' Laptop LCD LED Screen B173RTN01.1 B173R...,111130.98,Shenzhen ZDH LCD Tech Co LTD Store,//www.aliexpress.com/store/2490041,,,,None


In [5]:
import csv
with open("Aliexpresslaptops.csv", 'r') as file:
        reader = csv.reader(file)

In [6]:
reader

In [7]:
dataset = pd.read_csv("Aliexpresslaptops.csv")

In [8]:
dataset

,name,prices,store_names,store_link,shipping_price,extra_discounts,item_sold,old_price
0,High-end Lenovo Laptop ThinkPad X1 Carbon 2022...,8195500,HUWEI Factory Store,//www.aliexpress.com/store/836146,Free shipping,NaN,6,NaN
1,Gaming Laptop 15.6 Inch AMD R3 2200U 32GB RAM ...,639134.271304346.61,BSLAY Official Store,//www.aliexpress.com/store/5839012,Free shipping,NaN,3,NaN
2,2022 Professional Lenovo Gaming Laptop Legion ...,6228580,HUWEI Factory Store,//www.aliexpress.com/store/836146,Free shipping,NaN,4,NaN
3,VCHANCE 17.3 Inch 1080p 144Hz Gaming Portable ...,324246.72,VCHANCE Official Store,//www.aliexpress.com/store/1102820440,NaN,Extra 5% off with coins,4,NaN
4,Gaming Laptop HP OMEN 9 13th Gen Intel Core i5...,9263728.79,Blue Ocean Laptop Store,//www.aliexpress.com/store/1102248573,NaN,NaN,2,NaN
...,...,...,...,...,...,...,...,...
613,"B125XTN01.0 fit 12.5""1366X768 HB125WX1-200 LP1...",65347.3,LCD Source Store,//www.aliexpress.com/store/1102460600,Free shipping,Extra 2% off with coins,NaN,NaN
614,LP156WFC(SP)(H1) 30PINS 60HZ 1920*1080 Lap...,76429.58,Stars LCD Screen Store,//www.aliexpress.com/store/1102685113,NaN,NaN,1,NaN
615,LP133X7 N2C2 fit TM133XG-07 Laptop LCD LED Dis...,64664.06,LCD Source Store,//www.aliexpress.com/store/1102460600,Free shipping,Extra 2% off with coins,NaN,NaN
616,17.3'' Laptop LCD LED Screen B173RTN01.1 B173R...,111130.98,Shenzhen ZDH LCD Tech Co LTD Store,//www.aliexpress.com/store/2490041,NaN,NaN,NaN,NaN


In [9]:
dataset.columns

Index(['name', 'prices', 'store_names', 'store_link', 'shipping_price',
       'extra_discounts', 'item_sold', 'old_price'],
      dtype='object')

In [12]:
shipping_column = ['name','prices','item_sold']
shipping = dataset[shipping_column].copy()

shipping_index_name = 'shipping_id'
shipping = shipping.reset_index()

shipping.fillna({'shipping_price': '0'}, inplace = True)

In [13]:
shipping

,index,name,prices,item_sold
0,0,High-end Lenovo Laptop ThinkPad X1 Carbon 2022...,8195500,6
1,1,Gaming Laptop 15.6 Inch AMD R3 2200U 32GB RAM ...,639134.271304346.61,3
2,2,2022 Professional Lenovo Gaming Laptop Legion ...,6228580,4
3,3,VCHANCE 17.3 Inch 1080p 144Hz Gaming Portable ...,324246.72,4
4,4,Gaming Laptop HP OMEN 9 13th Gen Intel Core i5...,9263728.79,2
...,...,...,...,...
613,613,"B125XTN01.0 fit 12.5""1366X768 HB125WX1-200 LP1...",65347.3,NaN
614,614,LP156WFC(SP)(H1) 30PINS 60HZ 1920*1080 Lap...,76429.58,1
615,615,LP133X7 N2C2 fit TM133XG-07 Laptop LCD LED Dis...,64664.06,NaN
616,616,17.3'' Laptop LCD LED Screen B173RTN01.1 B173R...,111130.98,NaN


In [16]:
products_column = ['name', 'prices','store_names','store_link']
products = dataset[products_column].copy().drop_duplicates().reset_index(drop=True)

products.index.name = 'product_id'
products = products.reset_index()

In [ ]:
shipping.to_csv(shipping)
products.to_csv('products.csv', index = False)

In [17]:
products

,product_id,name,prices,store_names,store_link
0,0,High-end Lenovo Laptop ThinkPad X1 Carbon 2022...,8195500,HUWEI Factory Store,//www.aliexpress.com/store/836146
1,1,Gaming Laptop 15.6 Inch AMD R3 2200U 32GB RAM ...,639134.271304346.61,BSLAY Official Store,//www.aliexpress.com/store/5839012
2,2,2022 Professional Lenovo Gaming Laptop Legion ...,6228580,HUWEI Factory Store,//www.aliexpress.com/store/836146
3,3,VCHANCE 17.3 Inch 1080p 144Hz Gaming Portable ...,324246.72,VCHANCE Official Store,//www.aliexpress.com/store/1102820440
4,4,Gaming Laptop HP OMEN 9 13th Gen Intel Core i5...,9263728.79,Blue Ocean Laptop Store,//www.aliexpress.com/store/1102248573
...,...,...,...,...,...
595,595,"B125XTN01.0 fit 12.5""1366X768 HB125WX1-200 LP1...",65347.3,LCD Source Store,//www.aliexpress.com/store/1102460600
596,596,LP156WFC(SP)(H1) 30PINS 60HZ 1920*1080 Lap...,76429.58,Stars LCD Screen Store,//www.aliexpress.com/store/1102685113
597,597,LP133X7 N2C2 fit TM133XG-07 Laptop LCD LED Dis...,64664.06,LCD Source Store,//www.aliexpress.com/store/1102460600
598,598,17.3'' Laptop LCD LED Screen B173RTN01.1 B173R...,111130.98,Shenzhen ZDH LCD Tech Co LTD Store,//www.aliexpress.com/store/2490041


In [21]:
discount_columns = ['name','extra_discounts','prices']
discount = dataset[discount_columns].copy()

discount.index.name = 'discount_id'
discount = discount.reset_index()


discount['extra_discounts'] = discount['extra_discounts'].str.extract('(\d+)%').astype(float)


In [23]:
sales_columns = ['name','prices','item_sold']
sales = dataset[sales_columns].copy()

sales.index.name = 'sales_id'
sales = sales.reset_index()

sales

,sales_id,name,prices,item_sold
0,0,High-end Lenovo Laptop ThinkPad X1 Carbon 2022...,8195500,6
1,1,Gaming Laptop 15.6 Inch AMD R3 2200U 32GB RAM ...,639134.271304346.61,3
2,2,2022 Professional Lenovo Gaming Laptop Legion ...,6228580,4
3,3,VCHANCE 17.3 Inch 1080p 144Hz Gaming Portable ...,324246.72,4
4,4,Gaming Laptop HP OMEN 9 13th Gen Intel Core i5...,9263728.79,2
...,...,...,...,...
613,613,"B125XTN01.0 fit 12.5""1366X768 HB125WX1-200 LP1...",65347.3,NaN
614,614,LP156WFC(SP)(H1) 30PINS 60HZ 1920*1080 Lap...,76429.58,1
615,615,LP133X7 N2C2 fit TM133XG-07 Laptop LCD LED Dis...,64664.06,NaN
616,616,17.3'' Laptop LCD LED Screen B173RTN01.1 B173R...,111130.98,NaN


In [24]:
products.to_csv('products.csv', index=False)
sales.to_csv('sales.csv',index=False)
shipping.to_csv('shipping.csv', index = False)
discount.to_csv('discount.csv',index=False)

In [25]:
db_params = {
    'username': 'postgres',
    'password': 'judithpearl',
    'host': 'localhost',
    'port': '5432',
    'database': 'Kune'
}

db_url = f"postgresql://{db_params['username']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}"

engine = create_engine(db_url)

with engine.connect() as connection:
    shipping.to_sql('Shipping', connection, index = False, if_exists='replace')
    products.to_sql('Products', connection, index = False, if_exists='replace')
    discount.to_sql('Discount', connection, index = False, if_exists='replace')
    sales.to_sql('Sales', connection, index = False, if_exists='replace')

print("Database created")

Database created
